In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Make plots look nicer
sns.set_theme(style="whitegrid", context="talk")
plt.rcParams["figure.figsize"] = (12, 6)
plt.rcParams["figure.dpi"] = 120

In [ ]:
df = pd.read_csv("Access_to_Everyday_Life_Dataset.csv")
df.head(5)

In [ ]:
df = df.rename(columns={
    "geometry/coordinates/0": "lon",
    "geometry/coordinates/1": "lat",
    "properties/attribute_id": "attribute_id",
    "properties/label_type": "label_type",
    "properties/neighborhood": "neighborhood",
    "properties/severity": "severity",
    "properties/is_temporary": "is_temporary",
    "geometry/type": "geometry_type",
    "type": "feature_type"
})

df.head()


In [ ]:
df["severity"] = pd.to_numeric(df["severity"], errors="coerce")

# Sometimes these come as TRUE/FALSE strings
if df["is_temporary"].dtype == "object":
    df["is_temporary"] = df["is_temporary"].astype(str).str.upper().map({"TRUE": True, "FALSE": False})

df[["severity", "is_temporary"]].dtypes


In [ ]:
(df.isna().mean().sort_values(ascending=False) * 100).round(2)


In [ ]:
import folium
from folium.plugins import HeatMap, MarkerCluster, MiniMap, Fullscreen, LocateControl, MeasureControl, Draw, Geocoder

center = [df["lat"].mean(), df["lon"].mean()]


In [ ]:
hospitals = pd.read_csv("Seattle_Hospitals.csv")
print("Shape:", hospitals.shape)
hospitals.columns

In [ ]:
hospitals_clean = hospitals.rename(columns={
    "FACILITY": "poi_name",
    "x": "lon",
    "y": "lat"
})

hospitals_clean["poi_type"] = "Hospital"
hospitals_clean["source"] = "Seattle Open Data"

hospitals_clean[["poi_name", "poi_type", "lat", "lon"]].head()

# Approximate Seattle bounding box from barrier data
lat_min, lat_max = df["lat"].min(), df["lat"].max()
lon_min, lon_max = df["lon"].min(), df["lon"].max()

lat_min, lat_max, lon_min, lon_max

hospitals_seattle = hospitals_clean[
    (hospitals_clean["lat"] >= lat_min) &
    (hospitals_clean["lat"] <= lat_max) &
    (hospitals_clean["lon"] >= lon_min) &
    (hospitals_clean["lon"] <= lon_max)
].copy()

print("Hospitals in Sidewalk coverage area:", hospitals_seattle.shape[0])
hospitals_seattle[["poi_name", "lat", "lon"]]



In [ ]:
!pip -q install pyproj

In [ ]:
from pyproj import Transformer
import numpy as np

In [ ]:
def transform_xy_to_lonlat(x, y, epsg_in):
    transformer = Transformer.from_crs(f"EPSG:{epsg_in}", "EPSG:4326", always_xy=True)
    lon, lat = transformer.transform(x, y)
    return lon, lat

# Original x/y from the raw file (NOT the renamed lat/lon)
x = hospitals["x"].astype(float).values
y = hospitals["y"].astype(float).values

# Try EPSG:3857 (Web Mercator)
lon_3857, lat_3857 = transform_xy_to_lonlat(x, y, 3857)

# Try EPSG:2285 (WA StatePlane North ftUS) — common for Seattle
lon_2285, lat_2285 = transform_xy_to_lonlat(x, y, 2285)

def score_candidate(lon, lat):
    # Score = how many points fall inside a reasonable Seattle-ish bounding box
    ok = (lon > -123.5) & (lon < -121.5) & (lat > 47.0) & (lat < 48.2)
    return ok.sum()

s3857 = score_candidate(lon_3857, lat_3857)
s2285 = score_candidate(lon_2285, lat_2285)

print("Seattle-ish points if EPSG:3857:", s3857)
print("Seattle-ish points if EPSG:2285:", s2285)

# Pick the better one
if s3857 >= s2285:
    hospitals_fixed = hospitals.copy()
    hospitals_fixed["lon"] = lon_3857
    hospitals_fixed["lat"] = lat_3857
    chosen = 3857
else:
    hospitals_fixed = hospitals.copy()
    hospitals_fixed["lon"] = lon_2285
    hospitals_fixed["lat"] = lat_2285
    chosen = 2285

print("Chosen EPSG:", chosen)

hospitals_clean = hospitals_fixed.rename(columns={"FACILITY": "poi_name"})
hospitals_clean["poi_type"] = "Hospital"
hospitals_clean["source"] = "Seattle Open Data"

hospitals_clean[["poi_name", "lat", "lon"]].head()


In [ ]:
lat_min, lat_max = df["lat"].min(), df["lat"].max()
lon_min, lon_max = df["lon"].min(), df["lon"].max()

hospitals_seattle = hospitals_clean[
    (hospitals_clean["lat"] >= lat_min) &
    (hospitals_clean["lat"] <= lat_max) &
    (hospitals_clean["lon"] >= lon_min) &
    (hospitals_clean["lon"] <= lon_max)
].copy()

print("Hospitals in Sidewalk coverage area:", hospitals_seattle.shape[0])
hospitals_seattle[["poi_name", "lat", "lon"]]


In [ ]:
import folium
from folium.plugins import MarkerCluster

center = [df["lat"].mean(), df["lon"].mean()]
m_check = folium.Map(location=center, zoom_start=12, tiles="CartoDB positron")

# Plot hospitals
for _, r in hospitals_seattle.iterrows():
    folium.Marker(
        [r["lat"], r["lon"]],
        popup=r["poi_name"],
        icon=folium.Icon(color="red")
    ).add_to(m_check)

# Plot a small sample of barriers
cluster = MarkerCluster().add_to(m_check)
sample_barriers = df.dropna(subset=["lat","lon"]).sample(300, random_state=1)
for _, b in sample_barriers.iterrows():
    folium.CircleMarker([b["lat"], b["lon"]], radius=2, fill=True).add_to(cluster)

m_check


In [ ]:
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate great-circle distance between two points on Earth (meters)
    """
    R = 6371000  # Earth radius in meters

    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)

    a = (
        np.sin(dphi / 2) ** 2
        + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda / 2) ** 2
    )
    return 2 * R * np.arcsin(np.sqrt(a))

In [ ]:
# Prepare barrier data
barriers = df.dropna(subset=["lat", "lon", "severity", "is_temporary"]).copy()

RADIUS_METERS = 200  # walking buffer

results = []

for _, h in hospitals_seattle.iterrows():
    dists = barriers.apply(
        lambda b: haversine(h["lat"], h["lon"], b["lat"], b["lon"]),
        axis=1
    )
    nearby = barriers[dists <= RADIUS_METERS]

    results.append({
        "poi_name": h["poi_name"],
        "poi_type": "Hospital",
        "barrier_count": len(nearby),
        "avg_severity": nearby["severity"].mean() if len(nearby) > 0 else 0,
        "pct_permanent": 1 - nearby["is_temporary"].mean() if len(nearby) > 0 else 0
    })

hospital_access = pd.DataFrame(results)

hospital_access["risk_score"] = (
    hospital_access["barrier_count"]
    * hospital_access["avg_severity"]
    * (1 + hospital_access["pct_permanent"])
)

hospital_access.sort_values("risk_score", ascending=False)


In [ ]:
#Chart A: Risk score by hospital
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10, 6))
sns.barplot(
    data=hospital_access.sort_values("risk_score", ascending=False),
    x="risk_score",
    y="poi_name"
)
plt.title("Hospital Accessibility Risk Score (200m radius)")
plt.xlabel("Risk Score")
plt.ylabel("")
plt.tight_layout()
plt.show()


In [ ]:
#Barrier count vs severity
plt.figure(figsize=(8, 6))
sns.scatterplot(
    data=hospital_access,
    x="barrier_count",
    y="avg_severity",
    size="risk_score",
    sizes=(50, 500),
    legend=False
)
plt.title("Barrier Density vs Severity Around Hospitals")
plt.xlabel("Number of Barriers (within 200m)")
plt.ylabel("Average Severity")
plt.show()


In [ ]:
#Add hospitals to existing super map
import numpy as np
import pandas as pd
import folium

from folium.plugins import (
    HeatMap, MarkerCluster, MiniMap, Fullscreen, LocateControl,
    MeasureControl, Draw, Geocoder
)
from folium import IFrame

# -----------------------------
# 0) Preserve your existing map
# -----------------------------
m_no_poi = m  # keeps a reference to your previous super map (no POIs)

# -----------------------------
# 1) Create a NEW map for POIs
# -----------------------------
center = [df["lat"].mean(), df["lon"].mean()]
m_poi = folium.Map(location=center, zoom_start=12, tiles="CartoDB positron")

# Global tools (UX polish)
MiniMap(toggle_display=True).add_to(m_poi)
Fullscreen(position="topright").add_to(m_poi)
LocateControl(auto_start=False).add_to(m_poi)
MeasureControl(position="topleft").add_to(m_poi)
Geocoder(collapsed=True, add_marker=True).add_to(m_poi)

Draw(
    export=True,
    filename="selected_area.geojson",
    position="topleft",
    draw_options={"polyline": False, "circle": False, "circlemarker": False}
).add_to(m_poi)

# ---------------------------------------
# 2) VIEW: Heatmap weighted by severity
# ---------------------------------------
heat_fg = folium.FeatureGroup(name="VIEW: Heatmap (severity)", show=False)
heat_data = df[["lat", "lon", "severity"]].dropna().values.tolist()
HeatMap(heat_data, radius=10, blur=15, max_zoom=13).add_to(heat_fg)
heat_fg.add_to(m_poi)

# ---------------------------------------
# 3) VIEW: Clustered markers (basic)
# ---------------------------------------
cluster_fg = folium.FeatureGroup(name="VIEW: Clustered markers (basic)", show=False)
cluster = MarkerCluster().add_to(cluster_fg)

sample_basic = df.dropna(subset=["lat", "lon"]).sample(min(3000, len(df)), random_state=7)
for _, r in sample_basic.iterrows():
    popup = (f"Type: {r['label_type']}<br>"
             f"Neighborhood: {r['neighborhood']}<br>"
             f"Severity: {r['severity']}<br>"
             f"Temporary: {r['is_temporary']}")
    folium.CircleMarker(
        location=[r["lat"], r["lon"]],
        radius=3,
        popup=popup,
        fill=True
    ).add_to(cluster)

cluster_fg.add_to(m_poi)

# ---------------------------------------------------
# 4) VIEW: Toggle layers (perm/temp/severe)
# ---------------------------------------------------
toggle_view = folium.FeatureGroup(name="VIEW: Toggle layers (perm/temp/severe)", show=False)

fg_perm = folium.FeatureGroup(name="Permanent (not temporary)")
fg_temp = folium.FeatureGroup(name="Temporary")
fg_severe = folium.FeatureGroup(name="Severe (severity ≥ 4)")

clean = df.dropna(subset=["lat","lon","severity","is_temporary"]).sample(min(5000, len(df)), random_state=7)

for _, r in clean.iterrows():
    popup = (f"Type: {r['label_type']}<br>"
             f"Neighborhood: {r['neighborhood']}<br>"
             f"Severity: {r['severity']}<br>"
             f"Temporary: {r['is_temporary']}")

    marker = folium.CircleMarker(location=[r["lat"], r["lon"]], radius=3, popup=popup, fill=True)

    if r["is_temporary"] == True:
        marker.add_to(fg_temp)
    else:
        marker.add_to(fg_perm)

    if r["severity"] >= 4:
        folium.CircleMarker(location=[r["lat"], r["lon"]], radius=4, popup=popup, fill=True).add_to(fg_severe)

fg_perm.add_to(toggle_view)
fg_temp.add_to(toggle_view)
fg_severe.add_to(toggle_view)
toggle_view.add_to(m_poi)

# ---------------------------------------------------
# 5) VIEW: Severity grid overlay (choropleth-like)
# ---------------------------------------------------
def make_grid(df_in, cell_size=0.003):
    d = df_in.dropna(subset=["lat","lon","severity"]).copy()
    d["gx"] = (d["lon"] / cell_size).astype(int)
    d["gy"] = (d["lat"] / cell_size).astype(int)

    agg = d.groupby(["gx","gy"]).agg(n=("severity","size"), avg_sev=("severity","mean")).reset_index()
    agg["lon_min"] = agg["gx"] * cell_size
    agg["lon_max"] = (agg["gx"] + 1) * cell_size
    agg["lat_min"] = agg["gy"] * cell_size
    agg["lat_max"] = (agg["gy"] + 1) * cell_size
    return agg

grid_fg = folium.FeatureGroup(name="VIEW: Severity grid overlay", show=False)
grid = make_grid(df, cell_size=0.003)

for _, r in grid.iterrows():
    if r["n"] < 5:
        continue
    opacity = min(0.85, max(0.1, (r["avg_sev"] - 1) / 4))
    folium.Rectangle(
        bounds=[[r["lat_min"], r["lon_min"]], [r["lat_max"], r["lon_max"]]],
        fill=True, fill_opacity=opacity, weight=0,
        popup=f"Count: {int(r['n'])}<br>Avg severity: {r['avg_sev']:.2f}"
    ).add_to(grid_fg)

grid_fg.add_to(m_poi)

# ---------------------------------------------------
# 6) VIEW: Barrier types (top 6)
# ---------------------------------------------------
types_view = folium.FeatureGroup(name="VIEW: Barrier types (top 6)", show=False)

clean2 = df.dropna(subset=["lat","lon","label_type","severity"])
top_types = clean2["label_type"].value_counts().head(6).index
type_layers = {t: folium.FeatureGroup(name=f"Type: {t}") for t in top_types}

sample_types = clean2[clean2["label_type"].isin(top_types)].sample(min(6000, len(clean2)), random_state=7)
for _, r in sample_types.iterrows():
    popup = (f"Type: {r['label_type']}<br>"
             f"Neighborhood: {r['neighborhood']}<br>"
             f"Severity: {r['severity']}")
    folium.CircleMarker(location=[r["lat"], r["lon"]], radius=3, popup=popup, fill=True).add_to(type_layers[r["label_type"]])

for fg in type_layers.values():
    fg.add_to(types_view)

types_view.add_to(m_poi)

# ---------------------------------------------------
# 7) POIs: Hospitals (with risk score)
# ---------------------------------------------------
hosp_fg = folium.FeatureGroup(name="POIs: Hospitals (risk score)", show=True)

# Use your already computed hospital_access + hospitals_seattle
# (Assumes they exist and are correct now)
for _, r in hospitals_seattle.iterrows():
    row = hospital_access[hospital_access["poi_name"] == r["poi_name"]].iloc[0]
    popup = f"""
    <b>{r['poi_name']}</b><br>
    Risk score: {row['risk_score']:.2f}<br>
    Barriers within 200m: {int(row['barrier_count'])}<br>
    Avg severity: {row['avg_severity']:.2f}<br>
    % permanent: {row['pct_permanent']:.0%}
    """
    folium.CircleMarker(
        location=[r["lat"], r["lon"]],
        radius=10,
        popup=popup,
        fill=True
    ).add_to(hosp_fg)

hosp_fg.add_to(m_poi)

# Zoom to hospitals so you can immediately confirm they're visible
if len(hospitals_seattle) > 0:
    m_poi.fit_bounds(hospitals_seattle[["lat","lon"]].values.tolist())

# ---------------------------------------------------
# 8) One LayerControl at the end
# ---------------------------------------------------
folium.LayerControl(collapsed=False).add_to(m_poi)

m_poi

